In [1]:
import substratum as ss
import plotly.express as px
from dubious.distributions import Normal, LogNormal, Uniform, Beta 
from dubious.core import Uncertain, Context
rng = ss.Generator.from_seed(0)


In [2]:
ctx = Context()

length = Uncertain(Normal(10, 1), ctx=ctx)
width  = Uncertain(LogNormal(mu=1.6, sigma=0.2), ctx=ctx)

length.corr(width, 0.9)

area = length*width

area.summary()
# z=area
# data = z.sample(200000,seed=11)
# fig = px.histogram(
#     data,
#     nbins=200,
# )

# fig.show()

# print(z._ctx.get_corr(length,width))

# print(
#     length.node_id,
#     width.node_id,

# )

Summary for type: Uncertain
---------------------------
mean      : 51.645
median    : 49.344
std       : 15.187
variance  : 235.702
q05 / q95 : 30.076 / 79.512
q25 / q75 : 40.435 / 60.277



In [3]:
from dubious.distributions import Normal
from dubious.core import Uncertain, Context

ctx = Context()

x = Uncertain(Normal(0, 1), ctx=ctx)
y = Uncertain(Normal(10, 2), ctx=ctx)
z = x * y + 3

ctx.freeze(20_000)
print("run1:", z.mean(), z.quantile(0.05))
print("run2:", z.mean(), z.quantile(0.05))  # same

print(x.frozen)

ctx.unfreeze()
print("run3:", z.mean(), z.quantile(0.05))  # likely slightly different


run1: 3.0 3.0
run2: 3.0 3.0
False
run3: 2.8543822591568317 -13.825589585285694


In [4]:
from dubious.distributions import Normal
from dubious.core import Uncertain, Context
from dubious.core import Sampler

ctx = Context()

x = Uncertain(Normal(0, 1), ctx=ctx)
y = Uncertain(Normal(10, 2), ctx=ctx)
z = x * y

sampler = Sampler()

for i in range(0,6):
    print(z.draw())

ctx.freeze(5)
for i in range(0,6):
    print(z.draw())
    z.ctx.redraw()

ctx.unfreeze()
for i in range(0,6):
    print(z.draw())


-14.81016760039253
-16.044645916437815
12.801875106632133
-5.563086165954994
3.4618452554860277
-5.486499403883455
9.249552580636177
12.993256116208476
13.744319419178833
5.957910590365535
10.632589981775316
9.249552580636177
11.09006951587851
10.559381898379243
-2.1061164199793576
0.7174973967619789
21.8969733210219
-0.3012147523235361


In [5]:
data = area.sample(200000)
fig = px.histogram(
    data,
    nbins=200,
)

fig.show()

In [6]:
#two non-gassian marginals correlated using copula
from dubious.distributions import Beta, LogNormal
from dubious.core import Uncertain, Context

ctx = Context()

conv = Uncertain(Beta(20,80), ctx=ctx)
traffic = Uncertain(LogNormal(8.0,0.4), ctx=ctx)

conv.corr(traffic, 0.7)

sales = conv * traffic

print(f"Mean: {sales.mean()}")
print(f"p10, p90: {sales.quantile(0.1)}, {sales.quantile(0.9)}")

data = sales.sample(50_000)
fig = px.histogram(
    data,
    nbins=200,
)

fig.show()

Mean: 682.9556148632868
p10, p90: 282.49895472166844, 1195.2049066217655


In [7]:
from dubious.distributions import Normal, Beta
from dubious.core import Uncertain, Context

ctx = Context()

#We can define distribution parameters with other distributions.
normal = Normal(10, 1)
beta = Beta(normal,1)

x = Uncertain(normal, ctx=ctx)
y = Uncertain(beta, ctx=ctx)

#Apply some arithmetic.
x = x*y

#We can also use uncertain distributions to define parameters.
normal3 = Normal(2, 3)
print(normal3.mean())


2.0


In [8]:
from dubious.distributions import Normal, Beta
from dubious.core import Uncertain, Context

ctx = Context()
ctx.freeze(5)

normal = Normal(10, 1)
x = Uncertain(normal, ctx=ctx)
# x.sample(5)
for i in range(0,7):
    print(x.redraw())

print(f"float: {float(x)}")

12.07764197096364
12.14735012020578
9.711252458471458
10.543381811977408
6.877964494206235
12.07764197096364
12.14735012020578
float: 12.14735012020578


In [11]:
import substratum as ss

a = ss.Array([2, 2], [1.0, 2.0, 3.0, 4.0])
b = ss.Array([2, 2], [5.0, 6.0, 7.0, 8.0])

print(f"a + b = {(a + b).tolist()}")
print(f"a * b = {(a * b).tolist()}")

a + b = [6.0, 8.0, 10.0, 12.0]
a * b = [5.0, 12.0, 21.0, 32.0]


In [ ]:
import substratum as ss

a = ss.Array([2, 2], [1.0, 2.0, 3.0, 4.0])
b = ss.Array([2, 2], [5.0, 6.0, 7.0, 8.0])

print(f"a + b = {(a @ b).tolist()}")

a + b = [19.0, 22.0, 43.0, 50.0]


ValueError: Matrix is not positive definite